#### Imports 

In [25]:
!python --version
!pip show pandas selenium numpy

Python 3.8.2
Name: pandas
Version: 1.1.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /home/e1four15f/.local/lib/python3.8/site-packages
Requires: python-dateutil, numpy, pytz
Required-by: 
---
Name: selenium
Version: 3.141.0
Summary: Python bindings for Selenium
Home-page: https://github.com/SeleniumHQ/selenium/
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: /home/e1four15f/.local/lib/python3.8/site-packages
Requires: urllib3
Required-by: 
---
Name: numpy
Version: 1.19.2
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /home/e1four15f/.local/lib/python3.8/site-packages
Requires: 
Required-by: pandas


In [12]:
import os
import time
import functools
from random import shuffle

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#### План

- Таблица предпочтений 
- Словарь станция-айди в яндекс.метро
- Брут форс перебором считаем в словарь расстояние в минутах до каждой из станций в таблице
- Наиболее близкая станция - берем ее предпочтение 
- Если одинаковые по времени - пишу "не определились"

In [13]:
# Таблица предпочтенй
preferences = pd.read_csv('data/preferences.csv', header=None, names=['Station', 'Likes'])
preferences['Line'] = [2, 2, 2, 5, 7, 10, 9, 1, 6, 2, 5]
preferences

,Station,Likes,Line
0,Сокол,Кофе,2
1,Маяковская,Чай,2
2,Речной вокзал,Кофе,2
3,Белорусская,Чай,5
4,Сходненская,Кофе,7
5,Люблино,Кофе,10
6,Пражская,Чай,9
7,Юго-Западная,Чай,1
8,Калужская,Кофе,6
9,Ховрино,Чай,2


#### Таблица Станция - ID в Яндекс.Метро

In [14]:
from metro_data import LINES, STATIONS

metro_schema = pd.DataFrame(STATIONS.values()).rename(columns={'line': 'line_id', 'name': 'station_name'})
metro_schema['station_id'] = STATIONS.keys()
metro_schema['line_name'] = metro_schema.line_id.map(LINES)
display(metro_schema.head())

# usefull mappings 
station_id2station_name = dict(metro_schema[['station_id', 'station_name']].values)
station_name2station_id = {v:k for k, v in station_id2station_name.items()}

,line_id,station_name,station_id,line_name
0,1,Бульвар Рокоссовского,1,Сокольническая линия
1,1,Черкизовская,2,Сокольническая линия
2,1,Преображенская площадь,3,Сокольническая линия
3,1,Сокольники,4,Сокольническая линия
4,1,Красносельская,5,Сокольническая линия


#### Запрос на Яндекс.Метро

In [29]:
!google-chrome --version

Google Chrome 85.0.4183.102 


In [7]:
# Скачиваем нужный chromedriver
!wget http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_linux64.zip \
    && unzip -o chromedriver_linux64.zip && rm chromedriver_linux64.zip
!./chromedriver --version

--2020-09-18 02:57:09--  http://chromedriver.storage.googleapis.com/85.0.4183.87/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.222.128, 2a00:1450:4010:c0b::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.222.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5359609 (5,1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   5,11M  8,29MB/s    in 0,6s    

2020-09-18 02:57:11 (8,29 MB/s) - ‘chromedriver_linux64.zip’ saved [5359609/5359609]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
ChromeDriver 85.0.4183.87 (cd6713ebf92fa1cacc0f1a598df280093af0c5d7-refs/branch-heads/4183@{#1689})


In [111]:
# Инициализация браузера, в котором будут вычисления
chrome_options = Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(f'{os.getcwd()}/chromedriver', options=chrome_options)

In [122]:
@functools.lru_cache()
def trace_metro_route(from_station_id: int, to_station_id: int) -> int:
    # загружаем страницу
    browser.get(f'https://yandex.ru/metro/moscow?from={from_station_id}&to={to_station_id}&route=0/') 
    time.sleep(1)
    
    # выполняем скрипт страницы (т.к. без его выполнения исходный код страницы
    # не содержит необходимую информацию)
    browser.execute_script('//yastatic.net/s3/front-maps-static/front-metro/2.11.9/build/index/_index.ru.js') 
    time.sleep(1)
    
    # информация о времени содержится в классе "route-list-item__time" в формате "≈ 10 мин."
    str_time = browser.find_element_by_class_name('masstransit-route-snippet-view__route-duration').text 
    print(f'Проезд до станции {station_id2station_name[to_station_id]} занимает {str_time}')
    
    # так как маршрут может занять больше часа, обрабатываем и такое время
    # ищем подстроку с числом и преобразуем его в формат int
    # TODO (v.karmazin) datetime?
    if ('м' in str_time)&('ч' in str_time):
        return (int(str_time[str_time.find('≈')+1:str_time.find('ч')-1])*60 
                + int(str_time[str_time.find('ч')+2:str_time.find('м')]))
    elif 'ч' in str_time: 
        return (int(str_time[str_time.find('≈')+1:str_time.find('ч')-1])*60)
    
    return int(str_time[str_time.find('≈')+1:str_time.find('м')])

In [127]:
def find_likes(new_station_id: int) -> str:
    '''
    brute force
    '''
    routes = preferences[['Station', 'Likes']]
    routes['time'] = routes.Station.apply(
        lambda station_name: trace_metro_route(new_station_id, station_name2station_id[station_name])
    )

    candidates = routes[routes.time == routes.time.min()]
    return candidates.Likes.value_counts(sort=True).index[0]

#### Интерфейс выбора станции

![Схема в 2018 году](http://news.metro.ru/18/mm20180830.jpg)



In [11]:
!pip install -q ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbextension", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/notebook/nbextensions.py", line 988, in start
    super(NBExtensionApp, self).start()
  File "/usr/local/lib/python3.6/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python3.6/dist-packages/notebook/nbextensions.py", line 896, in start
    self.toggle_nbextension_python(self.extra_args[0])
  File "/usr/local/lib/python3.6/dist-p

,line_id,station_name,station_id,line_name
0,1,Бульвар Рокоссовского,1,Сокольническая линия
1,1,Черкизовская,2,Сокольническая линия
2,1,Преображенская площадь,3,Сокольническая линия
3,1,Сокольники,4,Сокольническая линия
4,1,Красносельская,5,Сокольническая линия
...,...,...,...,...
244,8,Петровский парк,245,Калининско-Солнцевская линия
245,2,Ховрино,246,Замоскворецкая линия
246,10,Окружная,247,Люблинско-Дмитровская линия
247,10,Верхние Лихоборы,248,Люблинско-Дмитровская линия


In [18]:
import ipywidgets as widgets

dropdown_line_selection = widgets.Dropdown(
    options=metro_schema.line_name.unique(),
    description='Линия метро:',
    style={'description_width': 'initial'}
)

dropdown_station_selection = widgets.Dropdown(
    options=metro_schema.station_name.values,
    description='Станция метро:',
    style={'description_width': 'initial'}
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        dropdown_station_selection.options = metro_schema[
            metro_schema.line_name == change['new']
        ].station_name.values

dropdown_line_selection.observe(on_change)

label_output = widgets.Label(style={'font_weight': 'bold'})

button_predict = widgets.Button(
    description='Предсказать',
    disabled=False,
    icon='check'
) 

def on_predicted(change):
    selected_station_name = dropdown_station_selection.value
    label_output.value = f'Предсказание для станции {selected_station_name}...'
    selected_station_preferences = find_likes(station_name2station_id[selected_station_name])
    results = f'Жители станции метро {selected_station_name} вероятно предпочитают {selected_station_preferences}'
    label_output.value = results

button_predict.on_click(on_predicted)

In [19]:
widgets.VBox([
    dropdown_line_selection,
    dropdown_station_selection,
    button_predict,
    label_output
])